# Artificial Intelligence :  Computer Assignment 2 - Genetic Algorithm  
> __Morteza Nouri, 810198481__

## Goals:
- How to formulate a problem to be solved using genetic algorithm
- Power of genetic algorithm to generate high-quality solutions to optimization and search problems
- How to choose appropriate heuristic function to select fittest individuals

## Description:
> In this project, we are given an encoded text which we want to find a key using genetic algorithm inorder to decode the text file.<br>
We use [__vigenere ciphering__](https://en.wikipedia.org/wiki/Vigen%C3%A8re_cipher) method to encryption and decryption. <br>
A genetic algorithm is a search heuristic that is inspired by Charles Darwin’s theory of natural evolution. This algorithm reflects the process of natural selection where the fittest individuals are selected for reproduction in order to produce offspring of the next generation.



### Part 0: Cleaning global text and creating dictionary

In this part the given global_text.txt preprocessed and then tokenized to create dictionary. we remove all stopwords, numbers, punctuations and whitespaces. (just keeping alphabets). this is done by __preprocess_text__ and __create_dictionary__ methods of __Decoder__ class.

### Part 1: Modeling

__Gene__: each alphabet. <br>
__Chromosome:__ a key which has 14 characters (genes).

### Part 2: Generating Initial population

In this part we generate a population of 100 chromosomes that each constructed chromosome with random characters. This part done with __generate_population(pop_size)__ method of __Decoder__ class. fitness of each chromosome set to 0.

### Part 3: Fitness Function

We decode the encoded_text with every chromosome of the population, and for each chromosme we count the number of matched tokens in dictionary, So this number will set the fitness of a chromosome.
This part done with __calculate_fitness_for_key(key)__ and __calculate_fitness(population)__ methods.


### Part 4: Crossover and Mutation

__Crossover__:
> This is the most significant phase in a genetic algorithm. For each pair of parents to be mated, a crossover point is chosen at random from within the genes. then new childs are created by exchanging the genes of parents among themselves until the crossover point is reached.<br>
This part done with __crossover(individuals, pop_size)__ and  __crossover_util(par1, par2)__ methods.<br>
__Note: in crossover method, we check the fitness of the new child, if it is lower than its parent, we don't add this to our population.__

__Mutation__: 
> Mutation occurs to maintain diversity within the population and prevent premature convergence.<br>
we mutate population with the probabilty of $0.2$. like the previous note, if the new mutated chromosme has a lower  fitness than its old, ignored.<br>
This part done with __mutation(population)__ method.


### Part 5: Implementation


In [1]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string, re, random

class Decoder:
    def __init__(self, global_text, encoded_text, key_lenght=14):
        self.global_text = global_text
        self.encoded_text = encoded_text
        self.key_lenght = key_lenght
        self.clean_encoded_text = self.__process_text(encoded_text)
        self.dictionary = self.__create_dictionary(global_text)
        self.n_encoded_token = len(self.clean_encoded_text.split())

    def decode(self):
        key = self.find_key()
        keyidx = 0
        plain_text = ''
        for c in self.encoded_text:
            if c.isalpha():
                if c.isupper():
                    plain_text += chr(((ord(c) - ord(key[keyidx % len(key)])) % 26) + 65)
                else:
                    plain_text += chr(((ord(c) - ord(key[keyidx % len(key)].lower())) % 26) + 97)
                keyidx += 1
            else:
                plain_text += c
        return plain_text

    def find_key(self):
        initial_population = self.__generate_population(100)
        population, max_fit = self.__calculate_fitness(initial_population)
        n = 0
        while  max_fit < self.n_encoded_token:
            n += 1
            # print(f'score: {self.__get_fittest_chromosome(population)[1]} gen: {n}')
            r, h = self.__selection(population)
            r = self.__crossover(r, len(r))
            population = list(r + h)
            population, max_fit = self.__calculate_fitness(population)
            population = self.__mutation(population)
        fittest_chromosome = self.__get_fittest_chromosome(population)
        print(f'Key: {fittest_chromosome[0].lower()} found in {n}th generation\n')
        return fittest_chromosome[0]

    def __selection(self, population):
        sorted_population = sorted(population, key=lambda pair: pair[1])
        remain = sorted_population[:int(len(population) * 0.6)]
        heros = sorted_population[-(len(population) -int(len(population) * 0.6)):]
        return remain, heros

    def __crossover(self, individuals, pop_size):
        next_generation = []
        random.shuffle(individuals)
        for i in range(pop_size // 2):
            a = individuals.pop()
            b = individuals.pop()
            c1, c2 = self.__crossover_util(a, b)
            if self.__calculate_fitness_for_key(c1[0]) < a[1]:
                c1 = a
            if self.__calculate_fitness_for_key(c2[0]) < b[1]:
                c2 = b
            next_generation.append(c1)
            next_generation.append(c2)
        return next_generation
        
    def __crossover_util(self, par1, par2):
        p1 = list(par1[0])
        p2 = list(par2[0])
        x = random.randint(0, self.key_lenght - 1)
        for i in range(x, self.key_lenght):
            p1[i], p2[i] = p2[i], p1[i]
        child1 = ''.join(p1)
        child2 = ''.join(p2)
        return (child1, par1[1]), (child2, par2[1]) 

    def __mutation(self, population):
        new_population = []
        for x in population:
            if random.random() < 0.2:
                old = x[:]
                p = random.randint(0, self.key_lenght -1)
                x = (x[0][:p] + random.choice(string.ascii_uppercase) + x[0][p + 1:], x[1])
                newfit = self.__calculate_fitness_for_key(x[0])
                x = (x[0], newfit)
                if newfit < self.__calculate_fitness_for_key(old[0]):
                    x = old
            new_population.append(x)
        return new_population
    
    def __get_fittest_chromosome(self, population):
        max_fit = -1
        max_chrom = ''
        for chromosome in population:
            if chromosome[1] > max_fit:
                max_fit = chromosome[1]
                max_chrom = chromosome
        return max_chrom
    
    def __generate_population(self, population_size):
        population = []
        for i in range(population_size):
            newChromosome = ''.join(random.choice(string.ascii_uppercase) for i in range(self.key_lenght))
            population.append((newChromosome, 0))
        return population

    def __calculate_fitness_for_key(self, key):
        fit = 0
        for token in self.__decrypt(key, self.clean_encoded_text):
            if token in self.dictionary:
                fit += 1
        return fit

    def __calculate_fitness(self, population):            
        max_fit = -1
        for i ,p in enumerate(population):
            fit = self.__calculate_fitness_for_key(p[0])
            population[i] = (p[0], fit)
            if fit > max_fit:
                max_fit = fit
        return population, max_fit
        
    def __decrypt(self,key, cipher_text):
        key = key.upper()
        keylen = len(key)
        keyidx = 0
        plain_text = []
        cipher_text = cipher_text.upper()
        for w in cipher_text.split():
            token = ''
            for i in range(0, len(w)):
                token += chr(((ord(w[i]) - ord(key[keyidx])) % 26) + 65)
                keyidx += 1
                if keyidx > keylen - 1:
                    keyidx = 0
            plain_text.append(token)
        return plain_text
        
    def __create_dictionary(self, text):
        text = self.__process_text(text)
        # Remove stopwords
        stop_words = set(stopwords.words("english"))
        tokens = word_tokenize(text)
        dictionary = [w for w in tokens if w not in stop_words]

        return set(dictionary)
    
    def __process_text(self, text):
        # UpperCase
        text = text.upper()

        text = re.sub('[^A-Z]+', ' ', text)

        # Remove whitespaces
        text = " ".join(text.split())

        return text

In [2]:
encodedText=open('encoded_text.txt').read()
globalText=open('global_text.txt').read()

d= Decoder(globalText,encodedText, key_lenght=14)
decodedText=d.decode()

Key: alberteinstein found in 311th generation



In [3]:
print(decodedText)

Albert Einstein
Old Grove Rd.
Nassau Point
Peconic, Long Island

August 2nd, 1939

F.D. Roosevelt,
President of the United States,
White House
Washington, D.C.

Sir:

Some recent work by E. Fermi and L. Szilard, which has been communicated to me in manuscript, leads me to expect that the element uranium may be turned into a new and important source of energy in the immediate future. Certain aspects of the situation which has arisen seem to call for watchfulness and, if necessary, quick action on the part of the Administration. I believe therefore that it is my duty to bring to your attention the following facts and recommendations:

In the course of the last four months it has been made probable—through the work of Joliot in France as well as Fermi and Szilard in America—that it may become possible to set up a nuclear chain reaction in a large mass of uranium by which vast amounts of power and large quantities of new radium-like elements would be generated. Now it appears almost certai

### Part 6: Answer of Questions

1- The use of smaller populations result in lower accuracy of the solution, obtained for a smaller computational time because of low diversity. The further increase of the population size increases the accuracy of solution but larger populations  does not improve the solution accuracy and only increase the needed computational resources because large diversity of chromosomes should be explored.

2- As mentioned in Q1, if the population size increases in each generation, more computational resources (time, memory) needed and accuracy doesn't improve.

3- <br>
_crossover:_ provide mixing of the solutions and convergence in a subspace.(convergence operator)<br>
_mutation:_ increases the diversity of the population and provides a mechanism for escaping from a local optimum.(divergence operator)<br>
If the crossover is not used, the population doesn't lead to converging thus the global optimum can't be found.
If the mutation is not used, the population get stuck in local optimum So we can't explore more space.<br>

4- The most significant part of GA is crossover. It increases the accuracy and converges the population to the goal state. Mutation happens in small probability but prevents from local optimum thus more states are explored.

5- In fact one of the indicators that the result is converging to a solution is repetition of population. But by increasing crossover probability and mutation probability we can explore more combinations of population.
Also we can record each tested individual and compare the regenerated individuals with the recorded ones to avoid any repetition but this method consumes many resources.

6- crossover, because converges the population.

7- In crossover and mutation, I compare new children with their parents and if a child has fitness lower than its parent I avoid adding this child to the population, So the population fitness rate increases and always next generation have fitness greater or equal than it previous generation. Also if we know some characters of the key, the state space decreases.